In [ ]:
import geopandas as gpd
import os
from datetime import date

SHARED_PROJECT_PATH = '...'
SHARED_PROJECT_PATH_POLY = '...'

In [ ]:
# Read EFM stations data
gdf_stations = gpd.read_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'EFM_stations', 'EFM_stoppoints.geojson'))

In [ ]:
# Read processed OSM raod network data
today = date.today().strftime('%d-%m-%Y')
gdf_graph_edges = gpd.read_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today, f'BP_safety-network_{today}.json'), driver='GeoJSON')

In [ ]:
# Add EFM station data to OSM road network as number of stations in a road segment buffer
gdf_graph_edges_buff = gdf_graph_edges.copy()
gdf_graph_edges_buff.geometry = gdf_graph_edges_buff.geometry.buffer(25, cap_style=2)
gdf_res = gpd.sjoin(gdf_graph_edges_buff[['gnx_edge_id', 'geometry']], gdf_stations[['TSYSSET', 'geometry']], how='inner', predicate='intersects')
gdf_graph_edges = gdf_graph_edges.merge(gdf_res.groupby('gnx_edge_id').size().to_frame().reset_index().rename(columns={0: 'nearby_station_no'}), on='gnx_edge_id', how='left')
gdf_graph_edges.fillna({'nearby_station_no': 0}, inplace=True)

In [ ]:
# Save road safety network with EFM station numbers as geojson
gdf_graph_edges.to_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today, f'BP_safety-network_{today}.json'), driver='GeoJSON')